# Correlation analysis

# 0. Imports

In [1]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
from covid_project.data_utils import (clean_covid_data, clean_policy_data,
                                      calculate_deltas, generate_state_case_dict)

In [2]:
# get the data
case_data = clean_covid_data()
policy_data = clean_policy_data()
# state_case_dict = generate_state_case_dict(case_data,
#                                            load_from_file=True
#                                            save_data=True)

In [3]:
correlated_df, state_cases_dict = calculate_deltas(
    case_df = case_data,
    policy_df = policy_data,
    measure_period = 14,
   # state_cases_dict = state_case_dict,
)

100%|██████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 528.28it/s]
0it [00:00, ?it/s]


series =  date
2020-01-22     0.000000
2020-01-23     0.000000
2020-01-24     0.000000
2020-01-25     0.000000
2020-01-26     0.000000
                ...    
2021-12-26     7.538067
2021-12-27    24.762551
2021-12-28    31.772953
2021-12-29    48.469772
2021-12-30    67.314940
Name: new_cases_7day_1e6, Length: 709, dtype: float64


IndexError: index 0 is out of bounds for axis 0 with size 0